*First explore the ColorDetection-2024.ipynb notebook

In [57]:
import cv2 # cv2 -> Imports open-cv our computer vision library
import numpy as np # Import the library numpy, and shorten it's name to np
import math

Define HSV Color range

In [58]:
# The low bounds, [h, s, v]
low_bounds = np.array([6, 180, 85])
# The top bounds, [h, s, v]
top_bounds = np.array([27, 255, 255])

In [59]:
image = cv2.imread("..\\TestingImages\\2023-Cone\\frame2.jpg") # Read the image from the file system

In [60]:
# Parameters: Name (can be anything), The actual image
cv2.imshow("Cone Image", image)

# Used to close the window when done
cv2.waitKey(0)
cv2.destroyAllWindows()

Create mask - Full explanation inside the ColorDetection-2024.ipynb notebook

In [61]:
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # Convert the image from BGR to HSV

# Create a mask using the low and top bounds
mask = cv2.inRange(hsv, low_bounds, top_bounds)

# Define the kernel for the morphological operation
# It is a 7*7 2d array of ones
# Try changing the size of the kernel to see how it affects the image
kernel = np.ones((7, 7), np.uint8)

# Perform morphological opening to remove noise
opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Perform morphological closing to close small holes
processed_mask = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

res = cv2.bitwise_and(image, image, mask=processed_mask)

# Display the processed mask
cv2.imshow("Processed Mask", processed_mask)
cv2.imshow("Result", res)

# Wait for a key press and close the mask window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [62]:
contours, _ = cv2.findContours(processed_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the image
for contour in contours:
    # draw the bounding rectangle around the contour
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)

# Show the image with contours
cv2.imshow("Contours", image)
# Wait for a key press and close the contours window
cv2.waitKey(0)
cv2.destroyAllWindows()

For non-symmetrical objects, where the yaw angle does matter, a bounding rectangle isn't enough.

Here is a simple way to give each rotation a unique value (Where similar rotations for similar positions are close to one another), originally developed by Sahar from the Poros Robotics team

In [63]:
def saharAlgorithm(contour, img):
    # Compute the center of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Find the point on the contour that is furthest from the center
    # It is almost always the cone top point
    # IMPORTANT: If the cone top is not visible in the image you will get undefined behavior
    distances = [np.linalg.norm(np.array([cX, cY]) - point[0]) for point in contour]
    max_distance_index = np.argmax(distances)
    furthest_point = tuple(contour[max_distance_index][0])

    X = furthest_point[0]
    Y = furthest_point[1]

    p1 = (cX, img.shape[0] - cY)
    p2 = (X, img.shape[0] - Y)

    # Difference in x coordinates
    dx = p2[0] - p1[0]

    # Difference in y coordinates
    dy = p2[1] - p1[1]

    # Angle between p1 and p2 in radians
    theta = math.atan2(dy, dx)

    angle = theta * 180 / math.pi

    if (dy < 0):
        angle = 360 + angle
    

    # Draw a circle at the center of the contour and a line from the center to the furthest point
    # This is only for debugging and testing
    # Visualize
    # cv2.circle(img, (cX, cY), 7, (255, 255, 255), -1)
    cv2.line(img, (cX, cY), furthest_point, (0, 0, 255), 2)

    return angle

In [64]:
saharAlgorithm(contours[0], image)
# Show the image with contours
cv2.imshow("Contours", image)
# Wait for a key press and close the contours window
cv2.waitKey(0)
cv2.destroyAllWindows()